I chose to go with the brute force method. Partly to illuminate how intractable this problem might become in 3-parameter space, and partly because I was pressed for time. I am willing to concede of course that the latter factored more heavily in my approach, however, this approach certainly did demonstrate how ill-suited brute force can be. If I were to encounter this problem in a different setting, I will most certainly not try to brute-force my way through it again. 

In [2]:
#########################################################################
#                                                                       #
# Import Kepler Data and Pick out points corresponding to 124 < t < 125 #
#                                                                       #
#########################################################################

import math
import numpy as np
import kplr
import matplotlib.pyplot as plt
%matplotlib notebook

# Find the target KOI.
client = kplr.API()
koi = client.koi(97.01)

# Get a list of light curve datasets.
lcs = koi.get_light_curves(short_cadence=False)

# Open the first dataset and read it
f = lcs[0].open()

hdu_data = f[1].data
time = hdu_data["time"]

index = np.where((time>124)&(time<125))
hdu = hdu_data[index]


flux_eclipse = hdu["sap_flux"]
time_eclipse = hdu["time"]
err_eclipse = hdu["sap_flux_err"]

f.close()

In [3]:
#########################################################################
#                                                                       #
# Perform two sigma exclusion 5 times to remove points 2 sigma away     #
# from the mean and calculate unobscured flux. Should get 33 entries    #
#                                                                       #
#########################################################################

non_transit = flux_eclipse

for i in range(5):
    mean = np.mean(non_transit)
    sigma = np.std(non_transit)

    twosigtest = abs(non_transit - mean)/(sigma)
    new_indices = np.where((twosigtest<2))
    non_transit = non_transit[new_indices]

unobscured_mean = np.mean(non_transit)

raw_Flux_Ratios = np.zeros(len(flux_eclipse))
for i in range(len(flux_eclipse)):
    raw_Flux_Ratios [i] = flux_eclipse[i] / unobscured_mean

print(len(non_transit))

33


In [60]:
########################################################################
#                                                                      #
#      Perform model calculations using scipy integration package      #
#                                                                      #
########################################################################

from scipy.integrate import quad

#vary tau
tau_low = 0.08
tau_high = 0.13
tau = 0.08
tau_step = 0.005

tau_values = np.zeros(int((tau_high - tau_low) / tau_step))

#vary p
p_low = 0.0575
p_high = 0.0975
p = 0.0575
p_step = 0.005

p_values = np.zeros(int((p_high - p_low) / p_step))

#vary t_0
t_0_low = 124.31
t_0_high = 124.71
t_0 = 124.31
t_0_step = 0.004

t_0_values = np.zeros(int((t_0_high - t_0_low) / t_0_step))

#define Delta function, specific intensity, numerator and denominator of flux ratio function
def Delta_Function(r,p,z): 
    r_squared = math.pow(r,2)
    z_squared = math.pow(z,2)
    p_squared = math.pow(p,2)
    if r >= z+p or r <= z-p:
        return 0 
    elif r + z <= p: 
        return 1
    else:
        return (math.pow(math.pi,-1)) * math.acos((z_squared - p_squared + r_squared) / (2*z*r))

def I(r):
    '''A Limb-darkening function'''
    mu = (1 - (r**2))**(0.5)
    return 1 - (1 - (mu**(0.5)))

def func1(r, p, z):
    return I(r) * (1 - Delta_Function(r,p,abs(z))) * 2 * r

def func2(r):
    return I(r) * 2 * r

#find model flux values
Model_Fluxes = np.zeros((int((tau_high - tau_low) / tau_step), int((p_high - p_low) / p_step), int((t_0_high - t_0_low) / t_0_step), len(flux_eclipse)))

for i in range(int((tau_high - tau_low) / tau_step)):
    tau_values[i] = tau
    p = 0.0575
    for j in range(int((p_high - p_low) / p_step)):
        p_values[j] = p
        t_0 = 124.31
        for k in range(int((t_0_high - t_0_low) / t_0_step)):
            t_0_values[k] = t_0
            
            #convert times in UTC to z (normalized separation of the centers)
            z_Kepler = np.zeros(len(flux_eclipse))

            for l in range(len(flux_eclipse)):
                z_Kepler[l] = (time_eclipse[l] - t_0) / tau
                z_l = z_Kepler[l]
                Model_Fluxes[i,j,k,l] = quad(func1, 0, 1, args = (p, z_l))[0]/(quad(func2, 0, 1)[0])
                
            t_0 += t_0_step
        p += p_step
    tau += tau_step

/home/patricia/anaconda3/lib/python3.6/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


In [61]:
print(np.shape(Model_Fluxes))

(10, 8, 99, 49)


In [62]:
########################################################################
#                                                                      #
#      Compute chi squared                                             #
#                                                                      #
########################################################################

chi_squared_values = np.zeros((int((tau_high - tau_low) / tau_step) * int((p_high - p_low) / p_step)* int((t_0_high - t_0_low) / t_0_step)))

index = 0

for i in range(int((tau_high - tau_low) / tau_step)):
    for j in range(int((p_high - p_low) / p_step)):
        for k in range(int((t_0_high - t_0_low) / t_0_step)):
            chi_squared = 0
            for l in range(len(flux_eclipse)):
                add = math.pow((raw_Flux_Ratios[l] - Model_Fluxes[i,j,k,l])/(err_eclipse[l]/unobscured_mean),2)
                chi_squared += add
            chi_squared_values[index] = chi_squared
            index += 1

In [63]:
########################################################################
#                                                                      #
#      Compute p values                                                #
#                                                                      #
########################################################################

nu = len(flux_eclipse) - 3
a = nu / 2
x = chi_squared_values / 2

def Incomplete_Gamma_Integrand(t,a): 
    return(math.exp(-t) * math.pow(t,a-1))

def p_value(a, x):
    return(quad(Incomplete_Gamma_Integrand, x, math.inf, args=(a))[0] / quad(Incomplete_Gamma_Integrand, 0, math.inf, args=(a))[0])

p = np.zeros(len(x))

for i in range(len(p)):
    p[i] = p_value(a,x[i])


In [66]:
########################################################################
#                                                                      #
#      Report best fit parameter set and uncertainty estimates         #
#                                                                      #
########################################################################

#since I recorded chi squared values as a 1-d array, to translate the position in the chi-squared array back to 
#the corresponding tau, p and t_0 parameter values requires some arithmetic.

num_tau_values = len(tau_values)
num_p_values = len(p_values)
num_t_0_values = len(t_0_values)

#find minimum chi squared value
min_chi = np.min(chi_squared_values)
min_chi_index = np.where(chi_squared_values == min_chi)
tau_loc = math.ceil(min_chi_index[0][0]/(num_p_values * num_t_0_values))
p_loc = math.ceil((min_chi_index[0][0] - ((num_p_values * num_t_0_values) * (tau_loc-1))) / num_t_0_values)
t_0_loc = min_chi_index[0][0]- ((num_p_values * num_t_0_values) * (tau_loc-1)) - (num_t_0_values * (p_loc-1))

min_tau = tau_values[tau_loc]
min_p = p_values[p_loc]
min_t_0 = t_0_values[t_0_loc]

epsilon = 300
min_chi_plus1 = min_chi + 1

min_chi_plus1_index = np.where((chi_squared_values <= min_chi_plus1 + epsilon)&(chi_squared_values >= min_chi_plus1 - epsilon))

tau_1_loc = math.ceil(min_chi_plus1_index[0][1]/(num_p_values * num_t_0_values))
p_1_loc = math.ceil((min_chi_plus1_index[0][1] - ((num_p_values * num_t_0_values) * (tau_1_loc-1))) / num_t_0_values)
t_0_1_loc = min_chi_plus1_index[0][1] - ((num_p_values * num_t_0_values) * (tau_1_loc-1)) - (num_t_0_values * (p_1_loc-1))

min_tau_plus1 = tau_values[tau_1_loc]
min_p_plus1 = p_values[p_1_loc]
min_t_0_plus1 = t_0_values[t_0_1_loc]

print("minimum chi value: " + str(min_chi))
print("minimum corresponding tau value: " + str(min_tau))
print("minimum corresponding p value: " + str(min_p))
print("minimum corresponding t_0 value: " + str(min_t_0))

print("minimum chi plus 1 value: " + str(min_chi_plus1))
print("corresponding tau value: " + str(min_tau_plus1))
print("corresponding p value: " + str(min_p_plus1))
print("corresponding t_0 value: " + str(min_t_0_plus1))

print("tau one sigma uncertainty: " + str(min_tau_plus1 - min_tau))
print("p one sigma uncertainty: " + str(min_p_plus1 - min_p))
print("t_0 one sigma uncertainty: " + str(min_t_0_plus1 - min_t_0))

minimum chi value: 88.3131637449
minimum corresponding tau value: 0.105
minimum corresponding p value: 0.0825
minimum corresponding t_0 value: 124.506
minimum chi plus 1 value: 89.3131637449
corresponding tau value: 0.105
corresponding p value: 0.0825
corresponding t_0 value: 124.502
tau one sigma uncertainty: 0.0
p one sigma uncertainty: 0.0
t_0 one sigma uncertainty: -0.004
